# Parte A.p3

# 1. Configuraciones iniciales

## 1.1 Import de librerias

In [74]:
import pandas as pd
from pandas import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from funpymodeling import status

In [75]:
pd.set_option('display.max_columns', None)

## 1.2) Lectura de datos

In [76]:
data = pd.read_csv('../data/customer_dataset.csv', sep=',')

In [77]:
data.head()

,orderAmount,orderState,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionAmount,transactionFailed,fraudulent,emailDomain,emailProvider,customerIPAdress_checked,sameCity
0,18.0,pending,True,card,JCB 16 digit,Citizens First Banks,18,False,False,com,yahoo,short_address,yes
1,26.0,fulfilled,True,bitcoin,VISA 16 digit,Solace Banks,26,False,False,com,yahoo,short_address,no
2,45.0,fulfilled,False,card,VISA 16 digit,Vertex Bancorp,45,False,True,com,yahoo,long_address,no
3,23.0,fulfilled,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.,23,False,True,com,yahoo,long_address,no
4,43.0,fulfilled,False,card,Mastercard,Vertex Bancorp,43,True,True,com,other,short_address,no


# 2) Discretización de variables

## 2.1) Misma Frecuencia y rango

In [78]:
tryout_data = data.copy()

In [79]:
# orderAmount

data['orderAmount'], saved_bins_order = pd.qcut(data['orderAmount'], q=5, duplicates='drop', retbins=True)

with open('../data/saved_bins_order.pickle', 'wb') as handle:
    pickle.dump(saved_bins_order, handle, protocol= pickle.HIGHEST_PROTOCOL)

In [80]:
# transactionAmount

data['transactionAmount'], saved_bins_transaction = pd.qcut(data['transactionAmount'], q=4, duplicates='drop', retbins=True)

with open('../data/saved_bins_transaction.pickle', 'wb') as handle:
    pickle.dump(saved_bins_transaction, handle, protocol= pickle.HIGHEST_PROTOCOL)

In [81]:
with open('../data/saved_bins_order.pickle', 'rb') as handle:
    new_saved_bins_order = pickle.load(handle)

with open('../data/saved_bins_transaction.pickle', 'rb') as handle:
    new_saved_bins_transaction = pickle.load(handle)

In [82]:
tryout_data['orderAmount'] = pd.cut(
    tryout_data['orderAmount'],
    bins = new_saved_bins_order,
    include_lowest= True
)

tryout_data.head()

,orderAmount,orderState,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionAmount,transactionFailed,fraudulent,emailDomain,emailProvider,customerIPAdress_checked,sameCity
0,"(9.999, 18.4]",pending,True,card,JCB 16 digit,Citizens First Banks,18,False,False,com,yahoo,short_address,yes
1,"(18.4, 30.0]",fulfilled,True,bitcoin,VISA 16 digit,Solace Banks,26,False,False,com,yahoo,short_address,no
2,"(39.0, 47.0]",fulfilled,False,card,VISA 16 digit,Vertex Bancorp,45,False,True,com,yahoo,long_address,no
3,"(18.4, 30.0]",fulfilled,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.,23,False,True,com,yahoo,long_address,no
4,"(39.0, 47.0]",fulfilled,False,card,Mastercard,Vertex Bancorp,43,True,True,com,other,short_address,no


# 3) Preaparación de datos

## 3.1) Interpretar valores

Comenzamos con paymentMethodIssuer

In [83]:
data['paymentMethodIssuer'].value_counts()

paymentMethodIssuer
Her Majesty Trust           43
Vertex Bancorp              37
Fountain Financial Inc.     35
His Majesty Bank Corp.      33
Bastion Banks               29
Bulwark Trust Corp.         29
Citizens First Banks        28
Grand Credit Corporation    27
Solace Banks                27
Rose Bancshares             25
B                            7
e                            5
c                            4
r                            3
                             2
n                            2
x                            2
o                            2
a                            1
p                            1
Name: count, dtype: int64

Se van a reemplazar los metodos de pago extraños por 'weird'

In [84]:
weird_payment_method = ["B", "e", "c", "r", " ", "n", "x", "o", "a", "p"]

for pay in weird_payment_method:
    data['paymentMethodIssuer'] = data['paymentMethodIssuer'].replace(pay, 'weird')

In [85]:
data['paymentMethodIssuer'].value_counts()

paymentMethodIssuer
Her Majesty Trust           43
Vertex Bancorp              37
Fountain Financial Inc.     35
His Majesty Bank Corp.      33
Bastion Banks               29
weird                       29
Bulwark Trust Corp.         29
Citizens First Banks        28
Solace Banks                27
Grand Credit Corporation    27
Rose Bancshares             25
Name: count, dtype: int64

Seguimos con paymentMethodProvider

In [86]:
data['paymentMethodProvider'].value_counts()

paymentMethodProvider
JCB 16 digit                   65
VISA 16 digit                  57
Voyager                        36
Diners Club / Carte Blanche    34
Maestro                        32
VISA 13 digit                  32
Discover                       25
American Express               22
JCB 15 digit                   20
Mastercard                     19
Name: count, dtype: int64

Seguimos con paymentMethodType

In [87]:
data['paymentMethodType'].value_counts()

paymentMethodType
card         242
apple pay     36
paypal        36
bitcoin       28
Name: count, dtype: int64

In [88]:
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,orderAmount,341,0.416361,0,0.000000,5,category
1,orderState,341,0.416361,0,0.000000,3,object
2,paymentMethodRegistrationFailure,477,0.582418,302,0.368742,2,object
3,paymentMethodType,477,0.582418,0,0.000000,4,object
4,paymentMethodProvider,477,0.582418,0,0.000000,10,object
5,paymentMethodIssuer,477,0.582418,0,0.000000,11,object
6,transactionAmount,0,0.000000,0,0.000000,4,category
7,transactionFailed,0,0.000000,609,0.743590,2,bool
8,fraudulent,0,0.000000,366,0.446886,2,bool
9,emailDomain,0,0.000000,0,0.000000,6,object


No va a ser necesario hacer un semaforo en fraudulent ya que le asignamos el fraudulent de los clientes a las transacciones, con el objetivo de tener un dataset completo de casos y posteriormente eliminando la información de los clientes.

# 4) Datos faltantes (categoricos)

En este caso es orderAmount la columna a tratar

In [89]:
data['orderAmount'].value_counts()

orderAmount
(30.0, 39.0]     98
(39.0, 47.0]     97
(9.999, 18.4]    96
(18.4, 30.0]     96
(47.0, 353.0]    91
Name: count, dtype: int64

In [ ]:
data['orderAmount'] = data['orderAmount'].cat.add_categories('unknown')

In [93]:
data['orderAmount'] = data['orderAmount'].fillna(value='unknown')
data['orderAmount'].value_counts()

orderAmount
unknown          341
(30.0, 39.0]      98
(39.0, 47.0]      97
(9.999, 18.4]     96
(18.4, 30.0]      96
(47.0, 353.0]     91
Name: count, dtype: int64

In [94]:
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,orderAmount,0,0.000000,0,0.000000,6,category
1,orderState,341,0.416361,0,0.000000,3,object
2,paymentMethodRegistrationFailure,477,0.582418,302,0.368742,2,object
3,paymentMethodType,477,0.582418,0,0.000000,4,object
4,paymentMethodProvider,477,0.582418,0,0.000000,10,object
5,paymentMethodIssuer,477,0.582418,0,0.000000,11,object
6,transactionAmount,0,0.000000,0,0.000000,4,category
7,transactionFailed,0,0.000000,609,0.743590,2,bool
8,fraudulent,0,0.000000,366,0.446886,2,bool
9,emailDomain,0,0.000000,0,0.000000,6,object


# 5) One Hot Encoding

In [106]:
cat = ['orderAmount','orderState', 'paymentMethodRegistrationFailure',
                       'paymentMethodType', 
                       'paymentMethodProvider',
                       'paymentMethodIssuer',
                       'emailDomain',
                       'emailProvider',
                       'customerIPAdress_checked',
                       'sameCity']


data_encoded = pd.get_dummies(data[cat])

In [107]:
others = ['transactionAmount', 'transactionFailed', 'fraudulent']
data_ohe = pd.concat([data[others], data_encoded], axis=1)

In [108]:
data_ohe_without_fraudulent = data_ohe.drop(["fraudulent"], axis=1)

In [113]:
data_ohe_without_fraudulent.columns

Index(['transactionAmount', 'transactionFailed', 'orderAmount_(9.999, 18.4]',
       'orderAmount_(18.4, 30.0]', 'orderAmount_(30.0, 39.0]',
       'orderAmount_(39.0, 47.0]', 'orderAmount_(47.0, 353.0]',
       'orderAmount_unknown', 'orderState_failed', 'orderState_fulfilled',
       'orderState_pending', 'paymentMethodRegistrationFailure_False',
       'paymentMethodRegistrationFailure_True', 'paymentMethodType_apple pay',
       'paymentMethodType_bitcoin', 'paymentMethodType_card',
       'paymentMethodType_paypal', 'paymentMethodProvider_American Express',
       'paymentMethodProvider_Diners Club / Carte Blanche',
       'paymentMethodProvider_Discover', 'paymentMethodProvider_JCB 15 digit',
       'paymentMethodProvider_JCB 16 digit', 'paymentMethodProvider_Maestro',
       'paymentMethodProvider_Mastercard',
       'paymentMethodProvider_VISA 13 digit',
       'paymentMethodProvider_VISA 16 digit', 'paymentMethodProvider_Voyager',
       'paymentMethodIssuer_Bastion Banks',
  

In [109]:
# Guardar las columnas de OHE para utilizar mas adelante
with open('../data/categories_ohe_without_fraudulent.pickle', 'wb') as handle:
    pickle.dump(data_ohe_without_fraudulent.columns, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 6) Dataset Encoded

In [115]:
filename = "../data/ohe_customer_dataset.csv"
data_ohe.to_csv(filename, index = False)